In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
scales = [7]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    filter = (x < 50)
    filtered_x = s.where(filter, x)
    return s.median(filtered_x)

error = 0.01
_, prover_model = computation_to_model(computation, precal_witness_path, True, selected_columns, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:34: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.tensor(True)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is depre

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 7, param_scale: 7, scale_input_multiplier: 1) ------------->

+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error | median_error | max_error | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0          | 0            | 0         | 0         | 0              | 0                | 0             | 0             | 0                  | 0                  | 0                      |
+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+---

scale:  [7]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":7,"param_scale":7,"scale_rebase_multiplier":1,"lookup_range":[-21758,19328],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":16232,"total_const_size":6612,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,7],"model_input_scales":[7],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":["ReLU",{"GreaterThan":{"a":0.0}},{"Floor":{"scale":128.0}}],"required_range_checks":[[-1,1],[-64,64]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715153521160}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, selected_columns, error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 8.326037883758545 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 40.0
==== Generating Proof ====
proof:  {'instances': [['8629d3dd63ed0085ee0b4d4877605e22c075aecd42b138aa05639b98cbb5961c', '0100000000000000000000000000000000000000000000000000000000000000', '0014000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x2682b0808a8f7410a29047d462d11fde0b8699f80df3912408d5c300e11eae0c16899510ca50653bf4b6dc7fd73985a19f62b708527a53dae92e3d9a3d70866f0fc65f3fb2149e05ce9cd80f5430004ebf0f5d44f297d2132206ed9e296cd4192676577e7039c6a781546847226853993d082c0c924e5681bf7b608549b49bed281dfb5ec0ee6996eee86e7b8e1a0d40eb29f03a412c45b1c75e541475927a421102773804411ce7652d5dc40cbdb5f20359195ee2a376aa3f9dfa7e17b91a5003c3e6ca3589f1342025ed775d5724b9c38400b3e572b723cbcd1fca2f117630111593240be79de3bf16c71a21c95f11e5548fa9c37dfa9e84fa150bb7a9a36410c4c8a27961a7e88ac80758554edc6cb6d428f6178efe3c27a65b575cf4023d267841e467c87f68

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [40.0]
